In [3]:
import subprocess
import pandas as pd
import numpy as np
import requests
import json
import os
import paho.mqtt.client as client
from pathlib import Path
from tqdm import tqdm
import time
import traceback

In [4]:
execure_url = 'http://0.0.0.0:5000/execute'
fetch_url = 'http://0.0.0.0:5000/fetch'
iot_app_path = "/home/asim/takshshila/IOT/IoTApp/app.py"
data_directory = "/home/asim/takshshila/IOT/Datasets/setup-stuff/gateway_and_dataset"
result_dir = "/home/asim/takshshila/IOT/Datasets/setup-stuff/gateway_and_dataset/Results"
algorithm = "function_approximation"

#net_sizes = [3,7]
#net_sizes = [4, 5, 6]
net_sizes = [3,5,7]
# net_sizes = [6,7]

# 200_20_4_5_6_0.85_3_0.04_[0.5, 0.075, 0.002]
# 200_18_3_7_0.90_3_0.009_[0.5, 0.075, 0.002]
number_of_shuffles = 4
convergence_threshold = 0.03 #0.000000002
num_chunks=3 # size of chunks 50*
data_passes=200 #number of chunks
chunk_passes= 15  # epochs
#learning_rates =[0.5,0.075,0.002,0.0005,0.00004, 0.000005] #[0.1, 0.075, 0.009] #[0.05, 0.002,0.0005]#[0.1, 0.075, 0.008]#[0.01, 0.075 ,0.002, 0.0005]#[0.002, 0.005, 0.0008] #[0.1, 0.05, 0.075]
learning_rates =[0.5,0.075,0.002, 0.0005]
partition_size = 100
sessionID= "06f0483a-987c-11ed-97d7-ebb1648f0903" #"5ec46db2-92b3-11ed-aaca-51e78ce51e2f"#"30929878-9043-11ed-a725-8326e5fa41d3"


In [5]:
prereqObj={
    "phase": "prereq",
    "mode":"mqtt",
    "algorithm": algorithm,
}

In [6]:
phase0Obj={
    "phase": "phase0",
    "num_chunks": num_chunks,
    "mode":"mqtt",
    "algorithm": algorithm,
    "session_id": sessionID
}



In [7]:
phase1Obj={
    "phase": "phase1",
    "data_passes": data_passes,
    "net_sizes": net_sizes,
    "chunk_passes": chunk_passes,
    "num_chunks": num_chunks,
    "learning_rates": learning_rates,
    "number_of_shuffles":number_of_shuffles,
    "num_feature_spaces": 1,
    "neigh_rate": 0.8,
    "train_test_split": 1,
    "phase3_passes": 1,
    "top_ranks": 2,
    "partition_size": partition_size,
    "neuron_init_criteria":"farthest_point",
    "algorithm": algorithm,
    "mode":"mqtt",
    "session_id": sessionID
}

In [8]:
phase2Obj = {
    "phase": "phase2",
    "data_passes": data_passes,
    "net_sizes": net_sizes,
    "chunk_passes": chunk_passes,
    "num_chunks": num_chunks,
    "learning_rates": learning_rates,
    "convergence_threshold":convergence_threshold,
    "num_feature_spaces": 1,
    "neigh_rate": 0.8,
    "train_test_split": 1,
    "phase3_passes": 1,
    "top_ranks": 100,
    "partition_size": 100,
    "neuron_init_criteria":"farthest_point",
    "mode":"mqtt",
    "algorithm": algorithm,
    "session_id": sessionID
}

In [9]:
phase3Obj = {
    "phase": "phase3",
    "data_passes": data_passes,
    "net_sizes": net_sizes,
    "chunk_passes": chunk_passes,
    "learning_rates": learning_rates,
    "convergence_threshold":convergence_threshold,
    "num_chunks": num_chunks,
    "num_feature_spaces": 1,
    "neigh_rate": 0.8,
    "train_test_split": 1,
    "phase3_passes": 1,
    "top_ranks": 2,
    "partition_size": 100,
    "mode":"mqtt",
    "algorithm": algorithm,
    "session_id": sessionID
}

In [10]:
def run_phase(algorithm, body):
    r = requests.post(execure_url, json=body)
    prereq_res = r.json()
    return prereq_res
    

In [11]:
def fetch_results(algorithm, phase, session_id):
    body = {"phase": phase, "algorithm": algorithm,"session_id": session_id}
    r = requests.post(fetch_url, json=body)
    prereq_res = r.json()
    return prereq_res

In [12]:
def process_results(data_dir, dataset, seed, split):
    proc, gateway_proc, details = None, None, {}
    try:
        gateway_proc = subprocess.Popen(['python', os.path.join(data_dir, 'gateway_simulation.py')])
        proc = subprocess.Popen(['python', os.path.join(data_dir, 'ECG_Stream_V2.py'), dataset, str(seed),split])

        #backend_proc = subprocess.Popen(['python3', iot_app_path])
        
        time.sleep(10)
        print("Dataset ### ", dataset)
        print("Prereq phase:::: ")
        prereqResult = run_phase(algorithm, prereqObj)
        print("Prereq result:::: ", prereqResult)
        sessionID = prereqResult['body']['session-id']
        time.sleep(10)
        
        print("Phase 0:::: ")
        phase0Obj['session_id'] = sessionID
        phase0Result = run_phase(algorithm, phase0Obj)
        time.sleep(10)
        
        print("Phase 1:::: ")
        phase1Obj['session_id'] = sessionID
        phase1Result =run_phase(algorithm, phase1Obj)
        time.sleep(10)
        
        print("Phase 2:::: ")
        phase2Obj['session_id'] = sessionID
        phase2Result =run_phase(algorithm, phase2Obj)
        time.sleep(10)
        
        print("Phase 3:::: ")
        phase3Obj['session_id'] = sessionID
        phase3Result = run_phase(algorithm, phase3Obj)
        time.sleep(10)
        
    
        print("Fetching Results :::: ")
        phase3Result=fetch_results(algorithm, "phase3", sessionID)
        if phase3Result['success']:
            details ={
                'dataset': dataset,
                'seed': seed,
                'session_Id':sessionID,
                'train_rmse': phase3Result['body']['train_error'],
                'valid_rmse': phase3Result['body']['validation_error'],
                'test_rmse': phase3Result['body']['test_error'],
                'best_score':phase3Result['body']['best_score'],
                'scores':phase3Result['body']['scores'],
                'val_rmses':phase3Result['body']['val_rmses'],
                'train_rmses':phase3Result['body']['train_rmses'],
                'test_rmses':phase3Result['body']['test_rmses'],
                
                'best_val_lr':phase3Result['body']['best_val_lr'],
                'best_fs_num_features':phase3Result['body']['best_fs_num_features'],
                'best_model_active_linear_weights':phase3Result['body']['best_model_active_linear_weights'],
                'best_model_active_gaussian_weights':phase3Result['body']['best_model_active_gaussian_weights'],
                'bound_hitting_gaussian_weights':phase3Result['body']['bound_hitting_gaussian_weights'],
                'neuron_centers':phase3Result['body']['neuron_centers'],
                'neuron_activity':phase3Result['body']['neuron_activity'],
                'radius_map':phase3Result['body']['radius_map'],
                'targets':phase3Result['body']['targets'],
                'predictions':phase3Result['body']['predictions'],
                'total_kernel_time':phase3Result['body']['total_kernel_time'],
                'total_host_time':phase3Result['body']['total_host_time'],
                'phase1_kernel_time':phase3Result['body']['phase1_kernel_time'],
                'phase1_host_time':phase3Result['body']['phase1_host_time'],
                'phase2_kernel_time':phase3Result['body']['phase2_kernel_time'],
                'phase2_host_time':phase3Result['body']['phase2_host_time'],
                'phase3_kernel_time':phase3Result['body']['phase3_kernel_time'],
                'phase3_host_time':phase3Result['body']['phase3_host_time'],
                'best_features':phase3Result['body']['best_feature_list'],
                'best_feature_names':phase3Result['body']['best_features_names'],
                'chunk_for_convergence':phase3Result['body']['chunk_for_convergence'],
                'val_data_size':phase3Result['body']['val_data_size'],
                'test_data_size':phase3Result['body']['test_data_size'],
            }
            
        time.sleep(10)

        backend_proc.kill()
        proc.kill()
        gateway_proc.kill()
    except:
        traceback.print_exc()
        if backend_proc:
            backend_proc.kill()
        if proc:
            proc.kill()
        if gateway_proc:
            gateway_proc.kill()
            
    return details


In [13]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_features','best_feature_names','train_rmse','valid_rmse','test_rmse','best_score','chunk_for_convergence','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','test_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions','val_data_size','test_data_size']

#datasets = ['Features_TestSet',"House_Price_Adv_Regression","Insurance","new_data_trans","OnlineNewsPopularity","ParkinsonData"]
#datasets = ["Telecom_data","arrhythmia","communities","dengue_features","Isolet"]
#datasets = ["yearMSD_new", "Sberbank_Russian_Housing_Market", "ENERGY_DATA_COMPLETE","blogData","slice_localization_data","ECG0_p02","Big_mart_sales"]
#datasets = ["yearMSD_new","Instant_Liking","new_data_trans","ParkinsonData","Telecom_data","arrhythmia","Big_mart_sales","Isolet","slice_localization_data","ECG0_p02"]
#datasets = ["slice_localization_data","ECG0_p02"]

#datasets = ["arrhythmia","new_data_trans","Telecom_data","Sberbank_Russian_Housing_Market","ENERGY_DATA_COMPLETE","Features_TestSet","ECG0_p02","Facebook_data","House_Price_Adv_Regression","Insurance","OnlineNewsPopularity","communities","dengue_features","blogData","Big_mart_sales"]

#datasets = ["arrhythmia","new_data_trans","Telecom_data","Sberbank_Russian_Housing_Market","ENERGY_DATA_COMPLETE"]
datasets = ["new_data_trans","Telecom_data","slice_localization_data","ECG0_p02","arrhythmia"]

seeds = [1, 50, 100, 150,200, 250, 300, 350, 400, 450]
#seeds = [50, 100, 150,200, 250,400, 450]

netsizeString = "_".join([str(val) for val in net_sizes])
split = "0.90"
for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    for seed in tqdm(seeds):
        result = process_results(data_directory, dataset,seed, split)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Dataset_{}_{}_{}_{}_{}_{}_{}_{}.csv".format(dataset, data_passes,chunk_passes,netsizeString, split, num_chunks,convergence_threshold, str(learning_rates)))
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df[df_column_order].to_csv(filepath, index=False)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

ECG connected


Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 387, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.8/http/client.py", line 1256, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/usr/lib/python3.8/http/clien

Dataset ###  new_data_trans
Prereq phase:::: 


NameError: name 'backend_proc' is not defined

In [23]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_features','best_feature_names','train_rmse','valid_rmse','test_rmse','best_score','chunk_for_convergence','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','test_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions','val_data_size','test_data_size']

#datasets = ['Features_TestSet',"House_Price_Adv_Regression","Insurance","new_data_trans","OnlineNewsPopularity","ParkinsonData"]
#datasets = ["Telecom_data","arrhythmia","communities","dengue_features","Isolet"]
#datasets = ["yearMSD_new", "Sberbank_Russian_Housing_Market", "ENERGY_DATA_COMPLETE","blogData","slice_localization_data","ECG0_p02","Big_mart_sales"]
#datasets = ["yearMSD_new","Instant_Liking","new_data_trans","ParkinsonData","Telecom_data","arrhythmia","Big_mart_sales","Isolet","slice_localization_data","ECG0_p02"]
#datasets = ["slice_localization_data","ECG0_p02"]

#datasets = ["arrhythmia","new_data_trans","Telecom_data","Sberbank_Russian_Housing_Market","ENERGY_DATA_COMPLETE","Features_TestSet","ECG0_p02","Facebook_data","House_Price_Adv_Regression","Insurance","OnlineNewsPopularity","communities","dengue_features","blogData","Big_mart_sales"]

#datasets = ["arrhythmia","new_data_trans","Telecom_data","Sberbank_Russian_Housing_Market","ENERGY_DATA_COMPLETE"]
datasets = ["Instant_Liking"]

seeds = [1, 50, 100, 150,200, 250, 300, 350, 400, 450]
#seeds = [50, 100, 150,200, 250,400, 450]

netsizeString = "_".join([str(val) for val in net_sizes])
split = "0.90"
for index, dataset in enumerate(datasets):
    df = pd.DataFrame()
    for seed in tqdm(seeds):
        result = process_results(data_directory, dataset,seed, split)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Dataset_{}_{}_{}_{}_{}_{}_{}_{}.csv".format(dataset, data_passes,chunk_passes,netsizeString, split, num_chunks,convergence_threshold, str(learning_rates)))
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df[df_column_order].to_csv(filepath, index=False)

  0%|                                                    | 0/10 [00:00<?, ?it/s]

ECG connected
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Dataset ###  Instant_Liking
Prereq phase:::: 
connecting to broker
Subscribing to all topics
adding topic to latest_elements 


127.0.0.1 - - [15/Mar/2023 09:23:15] "POST /execute HTTP/1.1" 200 -


...

Phase 3:::: 
Class: NeuronTrainingMeta
Operation: Neuron training kernel execution


Class: Mediator
Operation: Configuring and executing Mediator


Class: DataNormalization
Operation: Configuring and executing chunk normalization


Class: DataNormalizationMeta
Operation: Data normalization object construction


Operation: Data stream injection
Mode: mqtt


Class: MQTT
Operation: Constructing MQTT client and fetching data
Fetch configs: {'operation': 'fetch_data', 'normalizer_task': 'normalize', 'algorithm': 'function_approximation', 'feature_normalizer': MinMaxScaler(), 'target_normalizer': MinMaxScaler(), 'mode': 'mqtt', 'client_name': 'ZCU', 'client_topic': 'from/zcu', 'subscription': 'from/gateway', 'message': 'send_train_data', 'num_chunks': 3}


Class: DataNormalizationMeta
Operation: normalizing chunk


Class: NeuronTrainingMeta
Operation: Neuron training kernel execution


Class: Mediator
Operation: Configuring and executing Mediator


Class: DataNormalization
Operation: Config

Operation: Calculate Prediction wrapper operation


Class: CalculatePrediction
Operation: Wrapper operation Results


Class: CalculatePredictionMeta
Operation: Calculate Prediction kernel object destruction


Class: Mediator
Operation: Configuring and executing Mediator


Class: KernelInjector
Operation: Kernel injection
Mode: calculate_prediction


Class: CalculatePrediction
Operation: Calculate Prediction wrapper object construction


Class: CalculatePredictionMeta
Operation: Calculate Prediction kernel object construction


Class: CalculatePrediction
Operation: Calculate Prediction wrapper operation


Class: CalculatePrediction
Operation: Wrapper operation Results


Class: CalculatePredictionMeta
Operation: Calculate Prediction kernel object destruction


Class: Mediator
Operation: Configuring and executing Mediator


Class: DataNormalization
Operation: Configuring and executing chunk normalization


Class: DataNormalizationMeta
Operation: Data normalization object construction


Op

127.0.0.1 - - [15/Mar/2023 10:37:39] "POST /execute HTTP/1.1" 200 -


Fetching Results :::: 


127.0.0.1 - - [15/Mar/2023 10:37:49] "POST /fetch HTTP/1.1" 200 -
100%|████████████████████████████████████████| 10/10 [1:14:56<00:00, 449.66s/it]


## Test Running only phase 3

In [10]:
def process_results_custom(data_dir, dataset, seed, split, sessionId):
    proc, gateway_proc, details = None, None, {}
    try:
        gateway_proc = subprocess.Popen(['python', os.path.join(data_dir, 'gateway_simulation.py')])
        proc = subprocess.Popen(['python', os.path.join(data_dir, 'ECG_Stream_V2.py'), dataset, str(seed),split])

        backend_proc = subprocess.Popen(['python3', iot_app_path])
        sessionID  = sessionId
        time.sleep(10)
        print("Dataset ### ", dataset)
#         print("Prereq phase:::: ")
#         prereqResult = run_phase(algorithm, prereqObj)
#         print("Prereq result:::: ", prereqResult)
#         sessionID = prereqResult['body']['session-id']
#         time.sleep(10)
        
#         print("Phase 0:::: ")
#         phase0Obj['session_id'] = sessionID
#         phase0Result = run_phase(algorithm, phase0Obj)
#         time.sleep(10)
        
        print("Phase 1:::: ")
        phase1Obj['session_id'] = sessionID
        phase1Result =run_phase(algorithm, phase1Obj)
        time.sleep(10)
        
        print("Phase 2:::: ")
        phase2Obj['session_id'] = sessionID
        phase2Result =run_phase(algorithm, phase2Obj)
        time.sleep(10)
        
        print("Phase 3:::: ")
        phase3Obj['session_id'] = sessionID
        phase3Result = run_phase(algorithm, phase3Obj)
        time.sleep(10)
        
    
        print("Fetching Results :::: ")
        phase3Result=fetch_results(algorithm, "phase3", sessionID)
        if phase3Result['success']:
            details ={
                'dataset': dataset,
                'seed': seed,
                'session_Id':sessionID,
                'train_rmse': phase3Result['body']['train_error'],
                'valid_rmse': phase3Result['body']['validation_error'],
                'test_rmse': phase3Result['body']['test_error'],
                'best_score':phase3Result['body']['best_score'],
                'scores':phase3Result['body']['scores'],
                'val_rmses':phase3Result['body']['val_rmses'],
                'train_rmses':phase3Result['body']['train_rmses'],
                'test_rmses':phase3Result['body']['test_rmses'],
                
                'best_val_lr':phase3Result['body']['best_val_lr'],
                'best_fs_num_features':phase3Result['body']['best_fs_num_features'],
                'best_model_active_linear_weights':phase3Result['body']['best_model_active_linear_weights'],
                'best_model_active_gaussian_weights':phase3Result['body']['best_model_active_gaussian_weights'],
                'bound_hitting_gaussian_weights':phase3Result['body']['bound_hitting_gaussian_weights'],
                'neuron_centers':phase3Result['body']['neuron_centers'],
                'neuron_activity':phase3Result['body']['neuron_activity'],
                'radius_map':phase3Result['body']['radius_map'],
                'targets':phase3Result['body']['targets'],
                'predictions':phase3Result['body']['predictions'],
                'total_kernel_time':phase3Result['body']['total_kernel_time'],
                'total_host_time':phase3Result['body']['total_host_time'],
                'phase1_kernel_time':phase3Result['body']['phase1_kernel_time'],
                'phase1_host_time':phase3Result['body']['phase1_host_time'],
                'phase2_kernel_time':phase3Result['body']['phase2_kernel_time'],
                'phase2_host_time':phase3Result['body']['phase2_host_time'],
                'phase3_kernel_time':phase3Result['body']['phase3_kernel_time'],
                'phase3_host_time':phase3Result['body']['phase3_host_time'],
                'best_features':phase3Result['body']['best_feature_list'],
                'best_feature_names':phase3Result['body']['best_features_names'],
                'chunk_for_convergence':phase3Result['body']['chunk_for_convergence'],
                'val_data_size':phase3Result['body']['val_data_size'],
                'test_data_size':phase3Result['body']['test_data_size'],
            }
            
        time.sleep(10)

        backend_proc.kill()
        proc.kill()
        gateway_proc.kill()
    except:
        traceback.print_exc()
        if backend_proc:
            backend_proc.kill()
        if proc:
            proc.kill()
        if gateway_proc:
            gateway_proc.kill()
            
    return details


In [16]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_features','best_feature_names','train_rmse','valid_rmse','test_rmse','best_score','chunk_for_convergence','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','test_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions','val_data_size','test_data_size']

datasets = ["House_Price_Adv_Regression"] 

sessionIDs = ["6faf6858-c011-11ed-9ba0-77e52cb62188","a0f4305c-c015-11ed-9ba0-77e52cb62188"]
df = pd.DataFrame()

seeds = [50, 200]
# seeds = [200]
netsizeString = "_".join([str(val) for val in net_sizes])
split = "0.85"
for index, dataset in enumerate(datasets):
    for seed, session in tqdm(zip(seeds, sessionIDs)):
        result = process_results_custom(data_directory, dataset,seed, split, session)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Test_{}_{}_{}_{}_{}_.csv".format(dataset, data_passes,chunk_passes,netsizeString, split))
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df[df_column_order].to_csv(filepath, index=False)

0it [00:00, ?it/s]

ECG connected
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Dataset ###  House_Price_Adv_Regression
Phase 1:::: 
connecting to broker
Subscribing to all topics
adding topic to latest_elements 
API called: /execute


Class: PhaseInjector
Operation: Phase execution injection
Phase map: {'phase': 'phase1', 'data_passes': 200, 'net_sizes': [4, 5, 6], 'chunk_passes': 20, 'num_chunks': 3, 'learning_rates': [0.5, 0.075, 0.002], 'number_of_shuffles': 4, 'num_feature_spaces': 1, 'neigh_rate': 0.8, 'train_test_split': 1, 'phase3_passes': 1, 'top_ranks': 2, 'partition_size': 100, 'neuron_init_criteria': 'farthest_point', 'algorithm': 'function_approximation', 'mode': 'mqtt', 'session_id': '6faf6858-c011-11ed-9ba0-77e52cb62188'}


Class: Phase1
Operation: Phase1 construction
Phase configs: {'phase': 'phase1', 'data_passes': 200, 'net_sizes': [4, 5, 6], 'chunk_passes': 20, 'num_chunks': 3, 'learning_rates': [0.5, 0.075, 0.002], 'number_of_shuffles': 4, 'num_feature_spaces': 1, 'neigh_rate': 0.8, 'train_test_split': 1, 'phase3_passes': 1, 'top_ranks': 2, 'pa

Operation: Configuring and executing Mediator


Class: DataNormalization
Operation: Configuring and executing chunk normalization


Class: DataNormalizationMeta
Operation: Data normalization object construction


Operation: Data stream injection
Mode: mqtt


Class: MQTT
Operation: Constructing MQTT client and fetching data
Fetch configs: {'operation': 'fetch_data', 'normalizer_task': 'normalize', 'algorithm': 'function_approximation', 'feature_normalizer': MinMaxScaler(), 'target_normalizer': MinMaxScaler(), 'mode': 'mqtt', 'client_name': 'ZCU', 'client_topic': 'from/zcu', 'subscription': 'from/gateway', 'message': 'send_train_data', 'num_chunks': 3}


Class: DataNormalizationMeta
Operation: normalizing chunk


Class: CalculateActivityMeta
Operation: Calculate activity kernel execution


Class: Mediator
Operation: Configuring and executing Mediator


Class: KernelInjector
Operation: Kernel injection
Mode: compute_distance


Class: ComputeDistance
Operation: Compute distance wrapper obj

...

/home/asim/takshshila/IOT/IoTApp/mediator_operations/neuron_center_init/neuron_center_init.py:104: ConvergenceWarning: Number of distinct clusters (13) found smaller than n_clusters (36). Possibly due to duplicate points in X.
  aKMeansObj = KMeans(n_clusters=aLargestNet, random_state=0).fit(iChunk[:, :aFeatures])
127.0.0.1 - - [14/Mar/2023 22:30:59] "POST /execute HTTP/1.1" 200 -


Phase 3:::: 
Operation: Neuron center init using farthest point


Class: NeuronTraining
Operation: Neuron training wrapper operation


Class: Mediator
Operation: Configuring and executing Mediator


Class: DataNormalization
Operation: Configuring and executing chunk normalization


Class: DataNormalizationMeta
Operation: Data normalization object construction


Operation: Data stream injection
Mode: mqtt


Class: MQTT
Operation: Constructing MQTT client and fetching data
Fetch configs: {'operation': 'fetch_data', 'normalizer_task': 'normalize', 'algorithm': 'function_approximation', 'feature_normalizer': MinMaxScaler(), 'target_normalizer': MinMaxScaler(), 'mode': 'mqtt', 'client_name': 'ZCU', 'client_topic': 'from/zcu', 'subscription': 'from/gateway', 'message': 'send_train_data', 'num_chunks': 3}


Class: DataNormalizationMeta
Operation: normalizing chunk


Class: NeuronTrainingMeta
Operation: Neuron training kernel execution


Class: Mediator
Operation: Configuring and executing Med

127.0.0.1 - - [14/Mar/2023 22:34:06] "POST /execute HTTP/1.1" 200 -


Fetching Results :::: 


127.0.0.1 - - [14/Mar/2023 22:34:16] "POST /fetch HTTP/1.1" 200 -
2it [19:10, 575.26s/it]


In [ ]:



df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_features','best_feature_names','train_rmse','valid_rmse','test_rmse','best_score','chunk_for_convergence','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','test_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions','val_data_size','test_data_size']

datasets = ["arrhythmia"] 

sessionIDs = ["032b36a4-c119-11ed-be9e-fd7b019a143e","1169684c-c11b-11ed-be9e-fd7b019a143e"]
df = pd.DataFrame()

seeds = [400, 450]
# seeds = [200]
netsizeString = "_".join([str(val) for val in net_sizes])
split = "0.90"
for index, dataset in enumerate(datasets):
    for seed, session in tqdm(zip(seeds, sessionIDs)):
        result = process_results_custom(data_directory, dataset,seed, split, session)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Test_{}_{}_{}_{}_{}_.csv".format(dataset, data_passes,chunk_passes,netsizeString, split))
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df[df_column_order].to_csv(filepath, index=False)

0it [00:00, ?it/s]

ECG connected
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [ ]:
phase3Result=fetch_results(algorithm, "phase3", "sessionId")

## Testing

In [ ]:
def storingPhase3(sessionId, dataset, seed):
    phase3Result=fetch_results(algorithm, "phase3", sessionID)
    if phase3Result['success']:
        details ={
            'dataset': dataset,
            'seed': seed,
            'session_Id':sessionID,
            'train_rmse': phase3Result['body']['train_error'],
            'valid_rmse': phase3Result['body']['validation_error'],
            'test_rmse': phase3Result['body']['test_error'],
            'best_score':phase3Result['body']['best_score'],
            'scores':phase3Result['body']['scores'],
            'val_rmses':phase3Result['body']['val_rmses'],
            'train_rmses':phase3Result['body']['train_rmses'],
            'test_rmses':phase3Result['body']['test_rmses'],

            'best_val_lr':phase3Result['body']['best_val_lr'],
            'best_fs_num_features':phase3Result['body']['best_fs_num_features'],
            'best_model_active_linear_weights':phase3Result['body']['best_model_active_linear_weights'],
            'best_model_active_gaussian_weights':phase3Result['body']['best_model_active_gaussian_weights'],
            'bound_hitting_gaussian_weights':phase3Result['body']['bound_hitting_gaussian_weights'],
            'neuron_centers':phase3Result['body']['neuron_centers'],
            'neuron_activity':phase3Result['body']['neuron_activity'],
            'radius_map':phase3Result['body']['radius_map'],
            'targets':phase3Result['body']['targets'],
            'predictions':phase3Result['body']['predictions'],
            'total_kernel_time':phase3Result['body']['total_kernel_time'],
            'total_host_time':phase3Result['body']['total_host_time'],
            'phase1_kernel_time':phase3Result['body']['phase1_kernel_time'],
            'phase1_host_time':phase3Result['body']['phase1_host_time'],
            'phase2_kernel_time':phase3Result['body']['phase2_kernel_time'],
            'phase2_host_time':phase3Result['body']['phase2_host_time'],
            'phase3_kernel_time':phase3Result['body']['phase3_kernel_time'],
            'phase3_host_time':phase3Result['body']['phase3_host_time'],
            'best_features':phase3Result['body']['best_feature_list'],
            'best_feature_names':phase3Result['body']['best_features_names'],
            'chunk_for_convergence':phase3Result['body']['chunk_for_convergence'],
            'val_data_size':phase3Result['body']['val_data_size'],
            'test_data_size':phase3Result['body']['test_data_size'],
        }
        
        return details
    

In [ ]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_features','best_feature_names','train_rmse','valid_rmse','test_rmse','best_score','chunk_for_convergence','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','test_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions','val_data_size','test_data_size']


dataset = "Insurance"
seeds = [50, 100]
sessionIds = ["066772b4-bf8a-11ed-9f97-b56d983807cd", "2f522840-978a-11ed-893b-4f1e32c0f7e9"]


df = pd.DataFrame()
netsizeString = "_".join([str(val) for val in net_sizes])
for seed, sessionId in zip(seeds, sessionIds):
    details = storingPhase3(sessionId, dataset, seed)
    df = df.append(details, ignore_index=True)
    filepath = Path(result_dir, dataset, "{}_{}_.csv".format(dataset, netsizeString))
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df[df_column_order].to_csv(filepath, index=False)

In [ ]:
prereqResult = run_phase(algorithm, prereqObj)
print("Prereq result:: ",prereqResult)
sessionID = prereqResult['body']['session-id']

In [ ]:
print("Phase 0 ")
phase0Obj['session_id'] = sessionID
phase0Result = run_phase(algorithm, phase0Obj)
time.sleep(10)

In [ ]:
sessionID

In [ ]:
print("Phase :1")
phase1Obj['session_id'] = sessionID
phase1Result =run_phase(algorithm, phase1Obj)
time.sleep(10)

In [ ]:
print("Phase 2 ")
phase2Obj['session_id'] = sessionID
phase2Result =run_phase(algorithm, phase2Obj)
time.sleep(10)

In [ ]:
# prereqResult = run_phase(algorithm, prereqObj)
# print("Prereq result:: ",prereqResult)
# sessionID = prereqResult['body']['session-id']
# time.sleep(10)
# print("Phase 0 ")
# phase0Obj['session_id'] = sessionID
# phase0Result = run_phase(algorithm, phase0Obj)
# time.sleep(10)

# print("Phase :1")
# phase1Obj['session_id'] = sessionID
# phase1Result =run_phase(algorithm, phase1Obj)
# time.sleep(10)
#sessionID="04228570-92cb-11ed-8131-97f9b566c791"

# print("Phase 2 ")
# phase2Obj['session_id'] = sessionID
# phase2Result =run_phase(algorithm, phase2Obj)
# time.sleep(10)

# print("Phase 3 ")
# phase3Obj['session_id'] = sessionID
# phase3Result = run_phase(algorithm, phase3Obj)
# time.sleep(10)

# print("Training Finished !!!")
# phase3Result=fetch_results(algorithm, "phase3", sessionID)

### Fetch Testing

In [ ]:
phase3Result=fetch_results(algorithm, "phase3", sessionID)
if phase3Result['success']:        
    details = {
        'dataset': 'Instant_Liking',
        'seed': 50, # dummy
        'session':sessionID,
        'train_rmse': phase3Result['body']['train_error'],
        'valid_rmse': phase3Result['body']['validation_error'],
        'test_rmse': phase3Result['body']['test_error'],
        'best_score':phase3Result['body']['best_score'],
        'scores':phase3Result['body']['scores'],
        'val_rmses':phase3Result['body']['val_rmses'],
        'train_rmses':phase3Result['body']['train_rmses'],
        'best_val_lr':phase3Result['body']['best_val_lr'],
        'best_fs_num_features':phase3Result['body']['best_fs_num_features'],
        'best_model_active_linear_weights':phase3Result['body']['best_model_active_linear_weights'],
        'best_model_active_gaussian_weights':phase3Result['body']['best_model_active_gaussian_weights'],
        'bound_hitting_gaussian_weights':phase3Result['body']['bound_hitting_gaussian_weights'],
        'neuron_centers':phase3Result['body']['neuron_centers'],
        'neuron_activity':phase3Result['body']['neuron_activity'],
        'radius_map':phase3Result['body']['radius_map'],
        'targets':phase3Result['body']['targets'],
        'predictions':phase3Result['body']['predictions'],
        'total_kernel_time':phase3Result['body']['kernel_time'],
        'total_host_time':phase3Result['body']['total_host_time'],
        'phase1_kernel_time':phase3Result['body']['phase1_kernel_time'],
        'phase1_host_time':phase3Result['body']['phase1_host_time'],
        'phase2_kernel_time':phase3Result['body']['phase2_kernel_time'],
        'phase2_host_time':phase3Result['body']['phase2_host_time'],
        'phase3_kernel_time':phase3Result['body']['phase3_kernel_time'],
        'phase3_host_time':phase3Result['body']['phase3_host_time'],
        'best_feature_order':phase3Result['body']['best_feature_order'],
        'best_feature_list':phase3Result['body']['best_feature_list'],
        
        }
    
    df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_feature_order','best_feature_list','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']
    df = pd.DataFrame()
    netsizeString = "_".join([str(val) for val in net_sizes])
    
    
    df = df.append(details, ignore_index=True)
    filepath = Path(result_dir, dataset, "Epochs_{}_seed_{}_{}_.csv".format(dataset, s, netsizeString))
    df[df_column_order].to_csv(filepath, index=False)


In [ ]:
phase3Result['body'].keys()

In [ ]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_feature_order','best_feature_list','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']

dataset ="Instant_Liking"
df = pd.DataFrame()
net_sizes = str(4)
seed = str(50)

df = df.append(details, ignore_index=True)
filepath = Path(result_dir, dataset, "Epochs_{}_seed_{}_{}_.csv".format(dataset, seed,net_sizes ))
# filepath.parent.mkdir(parents=True, exist_ok=True)
df[df_column_order].to_csv(filepath, index=False)

In [ ]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_feature_order','best_feature_list','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']

dataset ="XOR_noiseless"
df = pd.DataFrame()
seeds = 34

df = df.append(details, ignore_index=True)
filepath = Path(result_dir, dataset, "Epochs_{}_seed_{}_.csv".format(dataset, seeds ))
filepath.parent.mkdir(parents=True, exist_ok=True)
# df.to_csv(filepath, index=False)
df[df_column_order].to_csv(filepath, index=False)

In [ ]:
phase3Result

#### Actual Training

In [ ]:

#df_column_order = ['dataset','seed','kernel_time','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_feature_order','best_feature_list','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']

# dataset, s = "Facebook", 50
#     # "Telecom_data", # -> done
#     # "yearMSD_new", # -> done
#     # "arrhythmia", # -> done
# #     # "Big_mart_sales",
#     # "blogData", # -> done
#     # "communities", # -> done
#     # "dengue_features", # -> done
#     # "ECG0_p02",
#     # "ENERGY_DATA_COMPLETE" # -> done
# ]
# seeds = [50]
datasets = ["House_Price_Adv_Regression", "Telecom_data", "Insurance","OnlineNewsPopularity"]
df = pd.DataFrame()
seeds = [50,100]
netsizeString = "_".join([str(val) for val in net_sizes])
for index, dataset in enumerate(datasets):
    for s in tqdm(seeds):
        result = process_results(data_directory, dataset,50,df)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Epochs_{}_seed_{}_{}_.csv".format(dataset, s, netsizeString))
        df[df_column_order].to_csv(filepath, index=False)

In [ ]:
df_column_order = ['dataset','seed','session_Id','total_kernel_time','total_host_time','best_feature_order','best_feature_list','train_rmse','valid_rmse','test_rmse','best_score','best_val_lr','best_fs_num_features','best_model_active_linear_weights','best_model_active_gaussian_weights','bound_hitting_gaussian_weights','scores','val_rmses','train_rmses','phase1_host_time','phase2_kernel_time','phase2_host_time','phase3_kernel_time','phase3_host_time','phase1_kernel_time', 'neuron_centers', 'neuron_activity', 'radius_map', 'targets', 'predictions']
# datasets = [
# #     "Facebook_data", # -> done
#     "Features_TestSet", # -> done
#     "House_Price_Adv_Regression", # -> done
#     "Instant_Liking",
#     "Insurance", # -> done
# #     # "Isolet" need to redo cuz of some error in RF,
#     # "new_data_trans", # -> done
#     # "OnlineNewsPopularity", # -> done
#     # "ParkinsonData", # -> done
#     # "Sberbank_Russian_Housing_Market", # -> done
#     # "slice_localization_data", # -> done
#     # "Telecom_data", # -> done
#     # "yearMSD_new", # -> done
#     # "arrhythmia", # -> done
# #     # "Big_mart_sales",
#     # "blogData", # -> done
#     # "communities", # -> done
#     # "dengue_features", # -> done
#     # "ECG0_p02",
#     # "ENERGY_DATA_COMPLETE" # -> done
# ]
# seeds = [50]
datasets = ["House_Price_Adv_Regression", "Instant_Liking", "Insurance","OnlineNewsPopularity"]
df = pd.DataFrame()
seeds = [50,100]
netsizeString = [str(val) for val in net]
for index, dataset in enumerate(datasets):
    for s in tqdm(seeds):
        result = process_results(data_directory, dataset,50,df)
        df = df.append(result, ignore_index=True)
        filepath = Path(result_dir, dataset, "Epochs_{}_seed_{}_.csv".format(dataset, s ))
        filepath.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath, index=False)
        #df[df_column_order].to_csv(filepath, index=False)
        #print(result)